**Predicting Credit Returns – An Asset Allocator’s Perspective**

Timing entry into different asset classes is the fundamental problem in tactical asset allocation. Investment grade and high yield bonds form a significant asset class with a particularly strong value anchor which makes timing the asset class a fruitful exercise. 

This article explores using the option adjusted spread (OAS) on credit indices as a timing signal into the asset class and as a predictor for future returns for both IG and HY indices.

The option adjusted spread (OAS) for a fixed income security measures the spread (or yield difference) between the fixed income security and the risk free yield, adjusted for any embedded options. Simply put – the OAS is the yield of the fixed income security above that of the equivalent risk free government bond, adjusted for any embedded optionality. This is the compensation that the bond holder gets for providing capital to the ‘credit-risky’ bond issuer compared to providing capital to the ‘risk-free’ government bond.

In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import os
from scipy.stats.stats import pearsonr
from zscore import zscore


**Reading in data**

Data is read wrangled and re-arranged in the following cells


In [7]:
endDt = pd.to_datetime('20200612', format='%Y%m%d')
stDt = pd.to_datetime('20000101', format='%Y%m%d')

# Read in Data
df_excess_ret = pd.read_csv(r'creditData_excess_ret.txt', sep = ' ')
df_excess_ret.columns = df_excess_ret.columns.str.replace('\t' , '')
df_excess_ret['dates'] = pd.to_datetime(df_excess_ret['dates'])
df_excess_ret = df_excess_ret.loc[(df_excess_ret['dates'] > stDt) & (df_excess_ret['dates']<= endDt)]
df_excess_ret.set_index('dates', inplace=True)
df_excess_ret = df_excess_ret.astype('float64')
df_excess_ret.info()

df_full_OAS = pd.read_csv(r'creditData_full_OAS.txt', sep = ' ', skiprows=range(1))
df_full_OAS.columns = df_full_OAS.columns.str.replace("\t","")
df_full_OAS['dates'] = pd.to_datetime(df_full_OAS['dates'])
df_full_OAS = df_full_OAS.loc[(df_full_OAS['dates'] > stDt) & (df_full_OAS['dates']<= endDt)]
df_full_OAS.set_index('dates', inplace=True)
df_full_OAS = df_full_OAS.astype('float64')
df_full_OAS.info()

df_OAS = pd.read_csv(r'creditData_OAS.txt', sep = '\s+', skiprows=range(1))
df_OAS.columns = df_OAS.columns.str.replace("\t","")
df_OAS['dates'] = pd.to_datetime(df_OAS['dates'])
df_OAS = df_OAS.loc[(df_OAS['dates'] > stDt) & (df_OAS['dates']<= endDt)]
df_OAS.set_index('dates', inplace=True)
df_OAS = df_OAS.astype('float64')
df_OAS.info()

df_full_MV = pd.read_csv(r'creditData_full_MV.txt', sep = ' ', skiprows=range(1))
df_full_MV.columns = df_full_MV.columns.str.replace("\t","")
df_full_MV['dates'] = pd.to_datetime(df_full_MV['dates'])
df_full_MV = df_full_MV.loc[(df_full_MV['dates'] > stDt) & (df_full_MV['dates']<= endDt)]
df_full_MV.set_index('dates', inplace=True)
df_full_MV = df_full_MV.astype('float64')
df_full_MV.info()



<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1067 entries, 2000-01-07 to 2020-06-12
Data columns (total 10 columns):
IGWDAll          1067 non-null float64
IGUSDAll         1067 non-null float64
IGGBPAll         1067 non-null float64
IGEURAll         1067 non-null float64
IGShortUSDAll    1063 non-null float64
IGShortGBPAll    733 non-null float64
IGShortEURAll    1063 non-null float64
HYWDAll          1067 non-null float64
HYUSDAll         1067 non-null float64
HYEURAll         1067 non-null float64
dtypes: float64(10)
memory usage: 91.7 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1067 entries, 2000-01-07 to 2020-06-12
Data columns (total 40 columns):
IGWDAAA          1067 non-null float64
IGWDAA           1067 non-null float64
IGWDA            1067 non-null float64
IGWDBBB          1067 non-null float64
IGUSDAAA         1067 non-null float64
IGUSDAA          1067 non-null float64
IGUSDA           1067 non-null float64
IGUSDBBB         1067 non-null float64
IGGBPAA

Looking at the option adjusted spread on Global Investment Grade and Global High Yield bond indices in Figures 1 and 2 respectively, we can clearly see that these are not constant through time as market and default risks and investor sentiment evolve. 

In [12]:
plt.plot(df_OAS['IGWDAll'])
plt.title('Investment Grade - Option Adjusted Spread')
plt.grid()
plt.ylabel('Basis Points (0.01%)')
plt.ylim([0, 500])
plt.axhline(np.mean(df_OAS['IGWDAll']),linestyle="dashed",linewidth= 1.5)


In [13]:
plt.plot(df_OAS['HYWDAll'])
plt.title('Global High Yield - Option Adjusted Spread')
plt.grid()
plt.ylabel('Basis Points (0.01%)')
plt.ylim([0, 2000])
plt.axhline(np.mean(df_OAS['HYWDAll']),linestyle="dashed",linewidth= 1.5)


**Adjusting the OAS for Differences in Composition Over Time**

In [9]:
colNames = [col for col in df_full_MV.columns if 'IGWD' in col]
WDIGIndicesMV = df_full_MV[colNames]
WDIGIndicesCurrentComposition=WDIGIndicesMV.iloc[-1,:].divide(WDIGIndicesMV.iloc[-1,:].sum(),axis=0)
WDIGIndicesComposition = WDIGIndicesMV.divide(WDIGIndicesMV.sum(axis=1),axis=0)
WDIGIndicesOAS = df_full_OAS[colNames]
WDIGCompositionAdjustedOAS = pd.DataFrame(np.sum(WDIGIndicesOAS*WDIGIndicesCurrentComposition,1), columns = ['IGWDAll'])

colNames = [col for col in df_full_MV.columns if 'IGUSD' in col]
USDIGIndicesMV = df_full_MV[colNames]
USDIGIndicesCurrentComposition=USDIGIndicesMV.iloc[-1,:].divide(USDIGIndicesMV.iloc[-1,:].sum(),axis=0)
USDIGIndicesComposition = USDIGIndicesMV.divide(USDIGIndicesMV.sum(axis=1),axis=0)
USDIGIndicesOAS = df_full_OAS[colNames]
USDIGCompositionAdjustedOAS = pd.DataFrame(np.sum(USDIGIndicesOAS*USDIGIndicesCurrentComposition,1), columns = ['IGUSDAll'])

colNames = [col for col in df_full_MV.columns if 'IGEUR' in col]
EURIGIndicesMV = df_full_MV[colNames]
EURIGIndicesCurrentComposition=EURIGIndicesMV.iloc[-1,:].divide(EURIGIndicesMV.iloc[-1,:].sum(),axis=0)
EURIGIndicesComposition = EURIGIndicesMV.divide(EURIGIndicesMV.sum(axis=1),axis=0)
EURIGIndicesOAS = df_full_OAS[colNames]
EURIGCompositionAdjustedOAS = pd.DataFrame(np.sum(EURIGIndicesOAS*EURIGIndicesCurrentComposition,1), columns = ['IGEURAll'])

colNames = [col for col in df_full_MV.columns if 'IGGBP' in col]
GBPIGIndicesMV = df_full_MV[colNames]
GBPIGIndicesCurrentComposition=GBPIGIndicesMV.iloc[-1,:].divide(GBPIGIndicesMV.iloc[-1,:].sum(),axis=0)
GBPIGIndicesComposition = GBPIGIndicesMV.divide(GBPIGIndicesMV.sum(axis=1),axis=0)
GBPIGIndicesOAS = df_full_OAS[colNames]
GBPIGCompositionAdjustedOAS = pd.DataFrame(np.sum(GBPIGIndicesOAS*GBPIGIndicesCurrentComposition,1), columns = ['IGGBPAll'])

colNames = [col for col in df_full_MV.columns if 'IGShortUSD' in col]
USDIGShortIndicesMV = df_full_MV[colNames]
USDIGShortIndicesCurrentComposition=USDIGShortIndicesMV.iloc[-1,:].divide(USDIGShortIndicesMV.iloc[-1,:].sum(),axis=0)
USDIGShortIndicesComposition = USDIGShortIndicesMV.divide(USDIGShortIndicesMV.sum(axis=1),axis=0)
USDIGShortIndicesOAS = df_full_OAS[colNames]
USDIGShortCompositionAdjustedOAS = pd.DataFrame(np.sum(USDIGShortIndicesOAS*USDIGShortIndicesCurrentComposition,1), columns = ['IGShortUSD'])

colNames = [col for col in df_full_MV.columns if 'IGShortEUR' in col]
EURIGShortIndicesMV = df_full_MV[colNames]
EURIGShortIndicesCurrentComposition=EURIGShortIndicesMV.iloc[-1,:].divide(EURIGShortIndicesMV.iloc[-1,:].sum(),axis=0)
EURIGShortIndicesComposition = EURIGShortIndicesMV.divide(EURIGShortIndicesMV.sum(axis=1),axis=0)
EURIGShortIndicesOAS = df_full_OAS[colNames]
EURIGShortCompositionAdjustedOAS = pd.DataFrame(np.sum(EURIGShortIndicesOAS*EURIGShortIndicesCurrentComposition,1), columns = ['IGShortEUR'])

colNames = [col for col in df_full_MV.columns if 'IGShortGBP' in col]
GBPIGShortIndicesMV = df_full_MV[colNames]
GBPIGShortIndicesCurrentComposition = GBPIGShortIndicesMV.iloc[-1,:].divide(GBPIGShortIndicesMV.iloc[-1,:].sum(),axis=0)
GBPIGShortIndicesComposition = GBPIGShortIndicesMV.divide(GBPIGShortIndicesMV.sum(axis=1),axis=0)
GBPIGShortIndicesOAS = df_full_OAS[colNames]
GBPIGShortCompositionAdjustedOAS = pd.DataFrame(np.sum(GBPIGShortIndicesOAS*GBPIGShortIndicesCurrentComposition,1), columns = ['IGShortGBP'])

colNames = [col for col in df_full_MV.columns if 'HYWD' in col]
WDHYIndicesMV = df_full_MV[colNames]
WDHYIndicesCurrentComposition=WDHYIndicesMV.iloc[-1,:].divide(WDHYIndicesMV.iloc[-1,:].sum(),axis=0)
WDHYIndicesComposition = WDHYIndicesMV.divide(WDHYIndicesMV.sum(axis=1),axis=0)
WDHYIndicesOAS = df_full_OAS[colNames]
WDHYCompositionAdjustedOAS = pd.DataFrame(np.sum(WDHYIndicesOAS*WDHYIndicesCurrentComposition,1), columns = ['HYWDAll'])

colNames = [col for col in df_full_MV.columns if 'HYUSD' in col]
USDHYIndicesMV = df_full_MV[colNames]
USDHYIndicesCurrentComposition=USDHYIndicesMV.iloc[-1,:].divide(USDHYIndicesMV.iloc[-1,:].sum(),axis=0)
USDHYIndicesComposition = USDHYIndicesMV.divide(WDHYIndicesMV.sum(axis=1),axis=0)
USDHYIndicesOAS = df_full_OAS[colNames]
USDHYCompositionAdjustedOAS = pd.DataFrame(np.sum(USDHYIndicesOAS*USDHYIndicesCurrentComposition,1), columns = ['HYUSDAll'])

colNames = [col for col in df_full_MV.columns if 'HYEUR' in col]
EURHYIndicesMV = df_full_MV[colNames]
EURHYIndicesCurrentComposition=EURHYIndicesMV.iloc[-1,:].divide(EURHYIndicesMV.iloc[-1,:].sum(),axis=0)
EURHYIndicesComposition = EURHYIndicesMV.divide(EURHYIndicesMV.sum(axis=1),axis=0)
EURHYIndicesOAS = df_full_OAS[colNames]
EURHYCompositionAdjustedOAS = pd.DataFrame(np.sum(EURHYIndicesOAS*EURHYIndicesCurrentComposition,1), columns = ['HYEURAll'])



In [10]:
OASCompAdjOAS = pd.concat([WDIGCompositionAdjustedOAS, USDIGCompositionAdjustedOAS,
                 GBPIGCompositionAdjustedOAS, EURIGCompositionAdjustedOAS, 
                 USDIGShortCompositionAdjustedOAS, GBPIGShortCompositionAdjustedOAS, EURIGShortCompositionAdjustedOAS,
                 WDHYCompositionAdjustedOAS, USDHYCompositionAdjustedOAS, EURHYCompositionAdjustedOAS], axis = 1, sort = False)
